In [ ]:
import os
import subprocess

# OpenFaceのディレクトリ
openface_dir = "/home/zhangzehang2/opencv-4.1.0/build/OpenFace/build/bin"

# 入力ビデオが格納されているディレクトリと、出力CSVを保存するディレクトリのパス
base_input_dir = "/home/zhangzehang2/emotion/video"
base_output_dir = "/home/zhangzehang2/emotion/video-csv1"

# Actorのフォルダリストを生成
actors = [f"Actor_{str(i).zfill(2)}" for i in range(1, 25)]

# 各アクターフォルダを処理
for actor in actors:
    input_actor_dir = os.path.join(base_input_dir, actor)
    output_actor_dir = os.path.join(base_output_dir, actor)

    # 出力ディレクトリが存在しない場合は作成
    if not os.path.exists(output_actor_dir):
        os.makedirs(output_actor_dir)

    # アクターフォルダ内の.mp4ファイルリストを取得
    video_files = [f for f in os.listdir(input_actor_dir) if f.endswith('.mp4')]

    # 各.mp4ファイルを処理
    for video_file in video_files:
        input_file_path = os.path.join(input_actor_dir, video_file)
        output_file_prefix = os.path.join(output_actor_dir, video_file.split('.')[0])

        # FeatureExtractionコマンドを実行
        command = f'cd {openface_dir} && ./FeatureExtraction -f "{input_file_path}" -of "{output_file_prefix}"'
        subprocess.call(command, shell=True)

print("All videos processed!")

In [ ]:
import os
import pandas as pd
import numpy as np

def extract_and_pad(df, max_frames=90):
    """Extract AUs features and pad with zeros if required."""
    features = df.iloc[:, 13:48].to_numpy()
    if len(features) < max_frames:
        padding = np.zeros((max_frames - len(features), 35))
        features = np.vstack((features, padding))
    return features[:max_frames]

input_dir = "/home/zhangzehang2/emotion/video-csv1"
male_output_dir = "/home/zhangzehang2/emotion/video-txt1/Male"
female_output_dir = "/home/zhangzehang2/emotion/video-txt1/Female"

# Ensure the output directories exist
os.makedirs(male_output_dir, exist_ok=True)
os.makedirs(female_output_dir, exist_ok=True)

for actor in os.listdir(input_dir):
    actor_path = os.path.join(input_dir, actor)
    if os.path.isdir(actor_path):
        if int(actor[-2:]) % 2 == 1:  # odd numbers
            output_dir = male_output_dir
        else:
            output_dir = female_output_dir
        
        for file in os.listdir(actor_path):
            if file.startswith('01') and file.endswith('.csv'):
                df = pd.read_csv(os.path.join(actor_path, file))
                data = extract_and_pad(df)
                
                output_file = os.path.join(output_dir, file.replace('.csv', '.txt'))
                np.savetxt(output_file, data, delimiter=',')

In [5]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler

def load_data(file_path):
    with open(file_path, 'r') as f:
        data = np.array([line.split(',') for line in f.read().splitlines()], dtype=float)
    return data

def load_and_combine_data(directories):
    combined_data = []
    for directory in directories:
        for file_name in os.listdir(directory):
            if file_name.endswith('.txt'):
                file_path = os.path.join(directory, file_name)
                data = load_data(file_path)
                combined_data.append(data)
    return np.vstack(combined_data)

def transform_and_normalize_data(input_path, output_path, scaler):
    # ファイルを読み込む
    data = load_data(input_path)

    # データを正規化
    normalized_data = scaler.transform(data)

    # データの転置
    data_transposed = normalized_data.T

    # 新しい形状でファイルに保存
    with open(output_path, 'w') as f:
        for row in data_transposed:
            f.write(' '.join(map(str, row)) + '\n')

def process_directory(source_directory, destination_directory, scaler):
    # 出力ディレクトリが存在しない場合は作成
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # フォルダ内のすべての.txtファイルを変更
    for file_name in os.listdir(source_directory):
        if file_name.endswith('.txt'):
            source_file_path = os.path.join(source_directory, file_name)
            destination_file_path = os.path.join(destination_directory, file_name)
            transform_and_normalize_data(source_file_path, destination_file_path, scaler)

# 男性と女性のディレクトリパス
directories = [
    '/home/zhangzehang2/emotion/video-txt1/Male',
    '/home/zhangzehang2/emotion/video-txt1/Female'
]

# 全データを読み込んで結合
combined_data = load_and_combine_data(directories)

# スケーラーを初期化してフィットさせる
scaler = StandardScaler().fit(combined_data)

# Maleディレクトリの処理
source_directory_male = '/home/zhangzehang2/emotion/video-txt1/Male'
destination_directory_male = '/home/zhangzehang2/emotion/video-txt4/Male'
process_directory(source_directory_male, destination_directory_male, scaler)

# Femaleディレクトリの処理
source_directory_female = '/home/zhangzehang2/emotion/video-txt1/Female'
destination_directory_female = '/home/zhangzehang2/emotion/video-txt4/Female'
process_directory(source_directory_female, destination_directory_female, scaler)


